In [ ]:
model_SVM = LinearSVC()
model_SVM = model_SVM.fit(X_train_scaled[features_SVM],y_train)

In [ ]:
train_scores_SVM = classification_metrics_training(model_SVM, "SVM", X_train_scaled[features_SVM], y_train, cv)
train_scores_SVM

## Algorithm comparison

### Metric comparison

#### Threshold = 0.5

In [ ]:
metrics_df = pd.concat([train_scores_logistic,train_scores_LDA,train_scores_QDA,train_scores_KNN_all,train_scores_KNN_numeric,train_scores_SVM],axis=0)
metrics_df

#### Empirical threshold

In [ ]:
metrics_df_empirical = pd.concat([train_scores_logistic_empirical,train_scores_LDA_empirical,train_scores_QDA_empirical,train_scores_KNN_all_empirical,train_scores_KNN_numeric_empirical],axis=0)
metrics_df_empirical

In [ ]:
from sklearn.metrics import roc_curve

calibrated_svc = CalibratedClassifierCV(model_SVM, method='sigmoid', cv='prefit')

def plot_roc(predict_fn, X, y, label=None):
    fprs, tprs, t = roc_curve(y, predict_fn(X)[:,-1])
    
    # Plot the ROC
    plt.plot(fprs, tprs, label="ROC "+label)
    plt.xlabel("FPR = 1 - specificity")
    plt.ylabel("TPR = sensitivity")
    plt.legend()

auc_list = []

for name, model, X in [('lr', model_logistic, X_train_scaled[features_logistic]), 
                    ('lda', model_LDA, X_train_scaled[features_LDA]), 
                    ('qda', model_QDA, X_train_scaled[features_QDA]),
                    ('SVC', calibrated_svc,X_train_scaled[features_SVM]),
                    ('KNN-all', model_KNN_all, X_train),
                    ('KNN-numeric', model_KNN_numeric, X_train[numeric_features])
                   ]:
    model.fit(X, y_train) # sistemare questa parte, fitta i modelli prima in modo che sia tutto ok
    plot_roc(model.predict_proba, X, y_train, name)
    y_prob_pred = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y_train, y_prob_pred)
    auc_list.append(auc)
auc_list